In [ ]:
import itertools
import subprocess
from collections import defaultdict
import math
import time

In [ ]:
var_dict = {}
rev_dict = {}
last_id = 1
product_num = 0

In [ ]:
def parse_edge_string(edge_string):
    edge_string = edge_string.strip().strip("e_{").strip("}")
    i, j = map(int, edge_string.split(","))
    return (i, j)

In [ ]:
def add_var_id(x, old_id=None):
    global last_id, var_dict, rev_dict
    xlast_id = 'x' + str(last_id)
    if old_id != None:
        var_dict[x] = old_id
        return
    var_dict[x] = xlast_id
    x = x + '}'
    x = x[0] + '_{' + x[1:]
    rev_dict[xlast_id] = x
    rev_dict['-' + xlast_id] = x
    last_id += 1
    return xlast_id

In [ ]:
# k clause must be true
def generate_clauses(edge, la, mu, triangles, angles):
    clauses = []
    clauses.append( f"-{mu} {edge} " + " ".join([f"-1 {x}" for x in angles]) + f" >= -{mu}" )
    clauses.append( f"-{la} {edge} " + " ".join([f"+1 {x}" for x in triangles]) + f" = 0" )
    return clauses

In [ ]:
def srg(n, K, la, mu):
    clauses = []
    
    for i, j in itertools.combinations(range(n), 2):
        edge_id = add_var_id(f"e{i},{j}")
        add_var_id(f"e{j},{i}", edge_id)
    for i in range(n):
        for j in range(i + 1, n):
            edge = var_dict[f"e{i},{j}"]
            la_clause = f"-{la} {edge}"
            mu_clause = f"+{mu} {edge}"
            for k in range(n):
                if k == i or k == j:
                    continue
                edge_1 = var_dict[f"e{i},{k}"]
                edge_2 = var_dict[f"e{k},{j}"]
                la_clause = la_clause + f" +1 {edge} {edge_1} {edge_2}"
                product_num += 1
                mu_clause = mu_clause + f" +1 {edge_1} {edge_2} -1 {edge} {edge_1} {edge_2}"
                product_num += 1
            la_clause += f" = 0"
            mu_clause += f" = {mu}"
            clauses.append(la_clause)
            clauses.append(mu_clause)
    # regularity
    for i in range(n):
        clause = ""
        for j in range(n):
            if j == i:
                continue
            e = var_dict[f"e{i},{j}"]
            clause = clause + f"+1 {e} "
        clause = clause + f"= {K}"
        clauses.append(clause)
    return clauses

In [ ]:
def clause_to_opb(clauses):
    # variables numbers, number of constraints, number of products, number of variables appearing in a prodcut
    pb_str = f"* #variable= {last_id - 1} #constraint= {len(clauses)} #product= {product_num} sizeproduct= {last_id - 1}\nmin: ;\n"
    print(last_id - 1, len(clauses))
    pb_str = pb_str + ';\n'.join(clauses) + ';\n'
    with open(f'srgs_opbs/NONLinearsrgN{N}K{K}L{LAMBDA}M{MU}.opb', 'w') as f:
        f.write(pb_str)
    print("should be done ...")
    return pb_str

In [ ]:
N, K, LAMBDA, MU = 27, 16, 10, 8
c = srg(N,K, LAMBDA, MU)
opb_file = clause_to_opb(c)

In [ ]:
def run_clasp(opb_file):
    with open(opb_file, "rb") as f:
        opb_contents = f.read()
    result = subprocess.run(['/usr/local/bin/clasp'], input=opb_contents, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    output = result.stdout.decode('utf-8')
    return output

In [ ]:
def parse_clasp_output(output):
    lines = output.strip().split("\n")
    solution = []
    for line in lines:
        if line.startswith("v"):
            variables = line[2:].split()
            for var in variables:
                solution.append(var)
    return solution

In [ ]:
# Run clasp and parse the output
start_time = time.time()
opb_file = f"srgs_opbs/srgN{N}K{K}L{LAMBDA}M{MU}.opb"
output = run_clasp(opb_file)
solution = parse_clasp_output(output)
edges_list = []
# Print the solution
if len(solution) == 0:
    print("UNSATISFIABLE")
else:
    for var in solution:
        if var.startswith('x'):
            res = rev_dict[var]
            if res.startswith('e'):
                edges_list.append(parse_edge_string(res))

In [ ]:
def check_srg(N, LAMBDA, MU, edges_list):
    adjacency_list = defaultdict(set)
    degrees = defaultdict(int)
    for u, v in edges_list:
        adjacency_list[u].add(v)
        adjacency_list[v].add(u)
        degrees[u] += 1
        degrees[v] += 1

    k = degrees[1]
    print("Regularity found is", k, "is ", K)
    if not all(degree == k for degree in degrees.values()):
        return False

    for u, v in edges_list:
        common_neighbors = adjacency_list[u] & adjacency_list[v]
        if len(common_neighbors) != LAMBDA:
            return False

    for u in range(N):
        for v in range(u + 1, N):
            if (u, v) not in edges_list and (v, u) not in edges_list:
                common_neighbors = adjacency_list[u] & adjacency_list[v]
                if len(common_neighbors) != MU:
                    return False

    return True

In [ ]:
def graph_to_csv(n, edges):
    header = ";" + ";".join(str(i) for i in range(n)) + "\n"
    row_strings = []
    for i in range(n):
        row = [str(i)] + ["0" if (i, j) not in edges and (j, i) not in edges else "1" for j in range(n)]
        row_strings.append(";".join(row) + "\n")
    csv_content = header + "".join(row_strings)
    with open(f"srgs_csvs/srgN{N}K{K}L{LAMBDA}M{MU}.csv", "w") as file:
        file.write(csv_content)

In [ ]:
if check_srg(N, LAMBDA, MU, edges_list):
    print("Result is strongly regular")
    graph_to_csv(N, edges_list)
elif len(edges_list) == 0:
    print("UNSATISFIABLE")
else:
    print("Wrong result has been generated.")
end_time = time.time()
execution_time = end_time - start_time
print(execution_time)